# Исследование надёжности заёмщиков для кредитного отдела банка

## Импорт библиотек

In [1]:
import pandas as pd
from pymystem3 import Mystem
from collections import Counter

## Изучение данных

In [2]:
clients_solvency = pd.read_csv('/datasets/data.csv')
clients_solvency.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
clients_solvency.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


>1. В таблице есть пропущенные значения типа NaN
>1. Смущает минусовой стаж и стаж в 4 строке не выглядит, как стаж в днях. Скорее всего, это ошибка. Для выполнения задания нам столбец со стажем не нужен, но возникает сомнение, что ошибки будут и в "нужных" нам столбцах. Стоит проверить столбцы children, family_status, total_income, purpose
>1. Есть разный регистр в столбце education
>1. Названия столбцов корректны и понятны, в исправлениях, считаю, что не нуждаются
>1. Значения отсутствуют только в столбцах days_employed и total_income. 
>1. Также данным в столбце total_income лучше присвоить целочисленное значение. Столбцу days_employed в принципе тоже не мешало бы, но т.к. он нам не нужен для решения задачи, не будем его трогать

## Предобработка данных

### Обработка пропусков

In [4]:
# Посмотрим на количество пустых значений в каждом столбце
clients_solvency.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

>Количество пропущенных значений в столбце days_employed равно количеству пропущенных значений в столбце total_income. Можно предположить, что это напрямую связано с тем, что при отсутствии стажа отсутствует также и доход. Это могут быть пенсионеры или студенты, не получающие доход и не работающие. Но это могут быть и люди других профессий, чей доход или стаж не указан по какой-либо определенной причине. Также стоит предположить, что даже если человек является пенсионером или студентом и не получает доход в качестве заработной платы, это не значит, что у него отсутствует доход в принципе. Пенсионеры получают пенсию, студенты могут получать стипендию или же помощь от родственников, также часть людей может работать неофициально. Следовательно, для решения задачи будет не верным заменить отсутствие дохода нулевым значением, поскольку по факту доход все равно присутствует. В этом случае было бы правильным заменить пропуски средним или медианой.

In [5]:
# проверим к каким типам занятости принадлежат пропуски
clients_solvency[(clients_solvency['total_income'].isnull() == True) & (clients_solvency['days_employed'].isnull() == True)]['income_type'].value_counts()

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64

> Пропуски относятся к разным типам занятости и пропуск данных скорее всего является ошибкой и может повлиять на решение задачи. Чтобы заменить пропуски, стоит посчитать среднее или медиану по каждой группе

In [6]:
# Проверим min и max по каждой группе в столбце total_income, чтобы понять заменять средним или медианой
clients_solvency_min_max = clients_solvency.groupby('income_type').agg({'total_income': ['min', 'max']}).reset_index()
display(clients_solvency_min_max)

income_type   total_income              
                              min           max
0      безработный   59956.991984  2.027225e+05
1        в декрете   53829.130729  5.382913e+04
2      госслужащий   29200.077193  9.104515e+05
3        компаньон   28702.812889  2.265604e+06
4        пенсионер   20667.263793  7.351033e+05
5  предприниматель  499163.144947  4.991631e+05
6        сотрудник   21367.648356  1.726276e+06
7          студент   98201.625314  9.820163e+04

>Данные по доходу очень разнятся, поэтому не совсем верно будет заменять пропущенные значения средним. В этом случае заменим пропущенные значения медианой по каждой нужной нам группе: сотрудник, компаньон, пенсионер, госслужащий, предприниматель

In [7]:
clients_solvency['total_income'] = clients_solvency.groupby('income_type')['total_income'].transform(lambda x: x.fillna(x.median()))
clients_solvency.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


> **Вывод:**   
избавились от пропущенных значений в столбце total_income. Теперь нужные нам столбцы пропущенных данных не содержат. С таблицей можно работать дальше

### Поиск и обработка странных значений

Выше мы планировали поискать странности в нужных нам столбцах: children, family_status, total_income, purpose. Вдруг там тоже будут ошибки по примеру ошибок в столбце со стажем

В столбце children числовые значения, поэтому если говорить о странностях можно предположить, что опять будут минусовые значения или же некорретные (кто-то мог проставить лишние цифры). Поищем странности

In [8]:
print(clients_solvency['children'].unique())

[ 1  0  3  2 -1  4 20  5]


>Странными выглядят значения -1 и 20. Остальное выглядит верным.

In [9]:
# Посмотрим столбец family_status
print(clients_solvency['family_status'].unique())

['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']


>В целом все хорошо, кроме регистра Н в последней категории. Приведем все к нижнему регистру

In [10]:
clients_solvency['family_status'] = clients_solvency['family_status'].str.lower()

In [11]:
print(clients_solvency['family_status'].unique())

['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'не женат / не замужем']


>Min и max по total_income считали ранее. В этом столбце все ок.

In [12]:
print(clients_solvency['purpose'].unique())

['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 'свадьба' 'получение дополнительного образования' 'покупка своего жилья'
 'операции с недвижимостью' 'получение высшего образования'
 'свой автомобиль' 'сделка с автомобилем' 'профильное образование'
 'высшее образование' 'покупка жилья для сдачи' 'на покупку автомобиля'
 'ремонт жилью' 'заняться высшим образованием']


>Каких-то ошибок или странностей нет, есть категории, которые скорее всего одни и те же и их надо объединить в одну.

>**Итого:** две явные странности - количество детей -1 и 20. Посчитаем количество таких странностей, чтобы понять влияют ли они на статистику

In [13]:
print("Количество 'children' -1 -", clients_solvency[clients_solvency['children'] == -1].count()[0])
print("Количество 'children' 20 -", clients_solvency[clients_solvency['children'] == 20].count()[0])

Количество 'children' -1 - 47
Количество 'children' 20 - 76


>Заемщиков со странностями = 123 человека. Это всего 0,57% от общего количества. Можно, конечно, предположить, что -1  - это лишний минус в начале и на самом деле у заемщика 1 ребенок. Сложнее предположить, что имелось ввиду под "20" - 2 или 0. А это принципиально для решения задачи. Поскольку суммарный процент заемщиков со "странным" количеством детей очень мал относительно всей базы, предлагаю просто не учитывать эти данные в выводах по детям. Считаю, что на статистику ошибочные данные не повлияют

### Замена типа данных

Заменим тип смущающий нас в самом начале тип данных в столбце total_income на целочисленный

In [14]:
clients_solvency['total_income'] = clients_solvency['total_income'].astype('int')
clients_solvency.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  int64  
 11  purpose           21525 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


>**Вывод**:  
заменили данные в столбце total_income на целочисленные. Можно было бы заменить данные также и в столбце days_employed, однако этот столбец нас не интересует в решении. Поэтому не будем терять на него время

### Обработка дубликатов

In [15]:
# Поищем дубликаты
clients_solvency.duplicated().sum()

54

In [16]:
#Посмотрим на эти дубликаты
clients_solvency[clients_solvency.duplicated()].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,NaN,41,среднее,1,женат / замужем,0,F,сотрудник,0,142594,покупка жилья для семьи
4182,1,NaN,34,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,142594,свадьба
4851,0,NaN,60,среднее,1,гражданский брак,1,F,пенсионер,0,118514,свадьба
5557,0,NaN,58,среднее,1,гражданский брак,1,F,пенсионер,0,118514,сыграть свадьбу
7808,0,NaN,57,среднее,1,гражданский брак,1,F,пенсионер,0,118514,на проведение свадьбы
8583,0,NaN,58,высшее,0,не женат / не замужем,4,F,пенсионер,0,118514,дополнительное образование
9238,2,NaN,34,среднее,1,женат / замужем,0,F,сотрудник,0,142594,покупка жилья для сдачи
9528,0,NaN,66,среднее,1,вдовец / вдова,2,F,пенсионер,0,118514,операции со своей недвижимостью
9627,0,NaN,56,среднее,1,женат / замужем,0,F,пенсионер,0,118514,операции со своей недвижимостью
10462,0,NaN,62,среднее,1,женат / замужем,0,F,пенсионер,0,118514,покупка коммерческой недвижимости


>**Вывод:**  
У нас нет ФИО заемщика или его идентификатора, столбец total_income (который мог бы, действительно, сильно разнится) в большинстве своем  - медиана, заполнившая пропуски, поэтому быть на 100% уверенными, что эти данные дублируются, мы не можем. Маловероятно, но, возможно, что часть из них, действительно, разные люди. В нашей ситуации потенциальных дубликатов всего 54 единицы. Думаю их удаление или не удаление не повлияют на выводы. Поэтому удалять в этом случае не будем.

### Лемматизация

Выше мы уже обратили внимание на то, что далеко не все цели кредита в столбце purpose являются на самом деле уникальными. Многие из них просто записаны по разному, что создаст путаницу в выборке и выводах. Надо разобраться с этим моментом

In [17]:
m = Mystem()
lemmas = []
for element in clients_solvency['purpose']:
    lemma = m.lemmatize(element)
    lemmas.extend(lemma)
#print(lemmas)

In [18]:
print(Counter(lemmas))

Counter({' ': 33677, '\n': 21525, 'недвижимость': 6367, 'покупка': 5912, 'жилье': 4473, 'автомобиль': 4315, 'образование': 4022, 'с': 2924, 'операция': 2610, 'свадьба': 2348, 'свой': 2235, 'на': 2233, 'строительство': 1881, 'высокий': 1375, 'получение': 1316, 'коммерческий': 1315, 'для': 1294, 'жилой': 1233, 'сделка': 944, 'дополнительный': 909, 'заниматься': 908, 'проведение': 777, 'сыграть': 774, 'сдача': 653, 'семья': 641, 'собственный': 635, 'со': 630, 'ремонт': 612, 'подержанный': 489, 'подержать': 479, 'приобретение': 462, 'профильный': 436})


>Итого получилось:  
>
>**цель 1**  - недвижимость (включая операции с ней)  
>**цель 2** - автомобиль  
>**цель 3** - образование  
>**цель 4** - свадьба  



In [19]:
clients_solvency['purpose_lemma'] = clients_solvency['purpose'].apply(m.lemmatize)
clients_solvency.head()
    

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemma
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]"
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]"
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]"
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]"
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]"


In [20]:
def purpose_grouped(row):
    if 'недвижимость' in row:
        return 'недвижимость'
    elif 'жилье' in row:
        return 'недвижимость'
    elif 'автомобиль' in row:
        return 'автомобиль'
    elif 'образование' in row:
        return 'образование'
    elif 'свадьба' in row:
        return 'свадьба'
    elif 'жилой' in row:
        return 'недвижимость'

clients_solvency['purpose_grouped'] = clients_solvency['purpose_lemma'].apply(purpose_grouped)
clients_solvency.head()
    

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemma,purpose_grouped
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]",недвижимость
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",автомобиль
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]",недвижимость
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]",образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]",свадьба


In [21]:
clients_solvency['purpose_grouped'].isnull().sum() # проверяем не появились ли пропуски

0

>**Вывод**  
Провели лемматизацию и выявили основные цели кредита

### Категоризация данных

Проведем категоризацию по столбцу children, чтобы ответить на вопрос: есть ли зависимость между наличием детей и возвратом кредита в срок

In [22]:
def children_grouped(row):
    children = row['children']
    if children == 0:
        return 'нет детей'
    elif 1 <= children <= 20: 
        return 'есть дети'
    
clients_solvency['children_grouped'] = clients_solvency.apply(children_grouped, axis=1)
clients_solvency.head()
        

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemma,purpose_grouped,children_grouped
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]",недвижимость,есть дети
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",автомобиль,есть дети
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]",недвижимость,нет детей
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]",образование,есть дети
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]",свадьба,нет детей


Категоризация по столбцу family_status, для ответа о взаимосвязи возврата кредита и семейного положения не требуется. В нем все понятно

Проведем категоризацию по столбцу total_income, чтобы ответить на вопрос про влияние уровня дохода на возврат кредита в срок
Какими данными о заработной плате пользоваться непонятно, т.к. не указан город банка, не указано чьи именно данные брать для категоризации людей по доходу и тп. Если это регионы - уровень заработной платы будет одним, если Москва - абсолютно другим. Поэтому возьмем примерные данные по рынку и данные Росстата

In [23]:
clients_solvency['total_income'] = clients_solvency['total_income'] / 1000
clients_solvency['total_income'].describe()

count    21525.000000
mean       165.224816
std         98.043668
min         20.667000
25%        107.798000
50%        142.594000
75%        195.549000
max       2265.604000
Name: total_income, dtype: float64

In [24]:
def total_income_grouped(row):
    total_income = row['total_income']
    if total_income < 107.798:
        return 'низкий доход'
    elif 107.798 <= total_income < 142.594:
        return 'ниже среднего'
    elif 142.594 <= total_income < 195.549:
        return 'средний доход'
    elif total_income >= 195.549:
        return 'высокий доход'
    
clients_solvency['total_income_grouped'] = clients_solvency.apply(total_income_grouped, axis=1)
clients_solvency.head()
    
    

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemma,purpose_grouped,children_grouped,total_income_grouped
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253.875,покупка жилья,"[покупка, , жилье, \n]",недвижимость,есть дети,высокий доход
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112.080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",автомобиль,есть дети,ниже среднего
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145.885,покупка жилья,"[покупка, , жилье, \n]",недвижимость,нет детей,средний доход
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267.628,дополнительное образование,"[дополнительный, , образование, \n]",образование,есть дети,высокий доход
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158.616,сыграть свадьбу,"[сыграть, , свадьба, \n]",свадьба,нет детей,средний доход


>**Вывод**  
Данные категоризированы 

## Определение зависимостей

### Зависимость между наличием детей и возвратом кредита в срок

In [25]:
pivot1 = clients_solvency.groupby('children_grouped').agg({'debt': ['count', 'sum']})
pivot1.columns = ['count', 'sum']
pivot1['ratio'] = pivot1['sum']/pivot1['count'] * 100
display(pivot1)

,count,sum,ratio
children_grouped,,,
есть дети,7329,677,9.237277
нет детей,14149,1063,7.512898


>**Вывод**  
Зависимость есть. Заемщики без детей возвращают кредиты лучше

### Зависимость между семейным положением и возвратом кредита в срок

In [26]:
pivot2 = clients_solvency.groupby('family_status').agg({'debt': ['count', 'sum']})
pivot2.columns = ['count', 'sum']
pivot2['ratio'] = pivot2['sum']/pivot2['count'] * 100
display(pivot2)

,count,sum,ratio
family_status,,,
в разводе,1195,85,7.112971
вдовец / вдова,960,63,6.562500
гражданский брак,4177,388,9.288963
женат / замужем,12380,931,7.520194
не женат / не замужем,2813,274,9.740491


>**Вывод**  
>
>Самый большой % людей с долгами - несостоящие в браке и состоящие в гражданском браке. Эти категории в принципе похожи между собой, поскольку понятие "гражданский брак" не имеет четких границ в определении. Лучше всего возвращают кредиты вдовцы. Интересно, что разведенные люди и вдовцы надежнее тех, кто состоит в браке.

### Зависимость между уровнем дохода и возвратом кредита в срок

In [27]:
pivot3 = clients_solvency.groupby('total_income_grouped').agg({'debt': ['count', 'sum']})
pivot3.columns = ['count', 'sum']
pivot3['ratio'] = pivot3['sum']/pivot3['count'] * 100
display(pivot3)

,count,sum,ratio
total_income_grouped,,,
высокий доход,5382,386,7.172055
ниже среднего,4419,385,8.712378
низкий доход,5381,427,7.935328
средний доход,6343,543,8.560618


>**Вывод**  
Лучший показатель по возвращаемости кредитов у людей с высоким доходом, худший - у людей со средним доходом и ниже среднего. 

### Зависимость между целями кредита и его возвратом в срок

In [28]:
pivot4 = clients_solvency.groupby('purpose_grouped').agg({'debt': ['count', 'sum']})
pivot4.columns = ['count', 'sum']
pivot4['ratio'] = pivot4['sum']/pivot4['count'] * 100
display(pivot4)

,count,sum,ratio
purpose_grouped,,,
автомобиль,4315,403,9.339513
недвижимость,10840,782,7.214022
образование,4022,370,9.199403
свадьба,2348,186,7.921635


>**Вывод**  
>
>Кредиты на недвижимость и все операции с ней самые возвращаемые. Практически рядом идут кредиты на свадьбу. Самый высокий процент должников, взявших кредит на автомобиль. Плюс стоит отметить, что чаще всего люди действительно берут кредиты на недвижимость, операции с ней и улучшение жилищных условий. 

## Результаты исследования

1. Люди с детьми возвращают кредиты хуже, чем люди, не имеющие детей.
2. Самый большой % людей с долгами - несостоящие в браке и состоящие в гражданском браке. 
3. Лучше всего возвращают кредиты вдовцы. Однако и по численности эта категория не велика. 
4. Говоря о семейном положении, стоит больше преимуществ отдавать людям, которые состоят в браке, нежели чем которые не состоят (если не включать сюда вдовцов и разведенных людей)  
5. Кредиты на недвижимость и все операции с ней - самые возвращаемые. Практически рядом идут кредиты на свадьбу. Самый высокий процент должников, взявших кредит на автомобиль. 
6. Лучший показатель по возвращаемости кредитов у людей с высоким доходом, худший - у людей со средним доходом и ниже среднего.
